Die ersten Transaktionen im Bitcoinnetzwerk waren so genannte P2PK-Transaktionen. Dabei wurde im für das Skript im Transaktionsoutput direkt der Public Key verwendet. Im Zuge einer chronologischen Reihenfolge wollen wir daher zunächst solch eine P2PK-Transaktion generieren und im Bitcoinnetzwerk verteilen.

In [4]:
from io import BytesIO

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2pk_script
from src.ecdsa import PrivateKey

1. Als erstes müssen wir einen Transaktion generieren welche unsere coins mit einem Public Key verknüpft. Dazu müssen zunächst alle benötigten Werte erzeugt werden um das "Formular" der Lockingtransaktion ausfüllen zu können.

In [6]:
private_key_wif = 'cQSweeycwBXRDfaRAULP72fnaobvZVkcQoQVhJpUPsMULxjK8oJK'
private_key_int = 123
version = 1
tx_id_to_spent = 'e0daceeaefa40ac7f3ff5298b9c3023a269a20a7aef38426d351af0cc3609649'
tx_index_to_spent = 0
script_pub_key_to_spent = '76a9148f39b19cb47396288b17c3a9f019119a140ba40088ac'
amount_to_spent = 9000
locktime = 0xffffffff

private_key_wif_receiver = 'cS1U7iREGpYmDuW7hRWGEyhirJudGptEwEDUmBKaPjeu9aRRSnxH'
private_key_receiver = PrivateKey(321)
public_key_receiver = private_key_receiver.get_public_key()


1a. Nun kann die Lockingtransaktion erzeugt und signiert werden. Um den Betrag direkt an einen public Key zu koppeln wird im Output der Transaktion ein ScriptPubKey der Form: public Key + CHECKSIG generiert.

In [9]:
script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(script_pub_key_to_spent)//2)[2:]}{script_pub_key_to_spent}")))
transaction_input = CTxIn(bytes.fromhex(tx_id_to_spent), tx_index_to_spent)


script_pubkey = p2pk_script(public_key_receiver.sec_format())
transaction_output = CTxOut(amount_to_spent, script_pubkey)

locking_transaction = CTx(version, [transaction_input], [transaction_output], locktime, is_testnet=True, is_segwit=False)

locking_transaction.sign_transaction(0, private_key_int, script_sig)

print(f"P2PK Lockingtransaktion: {locking_transaction.serialize_transaction().hex()}")

P2PK Lockingtransaktion: 0100000001499660c30caf51d32684f3aea7209a263a02c3b99852fff3c70aa4efeacedae0000000006c493046022100b7bd22555eb1f6a65f11fc7a96c2c0a4ffc89a5f17722d6eb69dbba0031ea97f022100c8e6af8737a84201b79e3446511a1d455af74f6f28d0a55d20f7af937e4b8b6d012103a598a8030da6d86c6bc7f2f5144ea549d28211ea58faa70ebf4c1e665c1fe9b5ffffffff0128230000000000002321038b4544fc1fdfa06e456c1115a1dc831c85e7f1c5e620eca51c20802d36a4bc6bacffffffff


2. Nun wollen wir die eben erstellte Transaktion wieder ausgeben. Dazu muss im Inputfeld (in dem auf den eben erzeugten Output referenziert wird), zum Beweis das wir berechtigt sind den Output auszugeben lediglich eine Signatur mit privaten Schlüssel des Empfängers der ersten Transaktion stehen. Der Einfachheit halber senden wir die neue Transaktion an den selben public Key wie die Erste. Bitte macht das NIEMALS NIEMALS NIEMALS für eine echte Transaktion auf der Mainchain!!!!!

In [11]:
version = 1
tx_id_to_spent = 'f60044de9eadb365c03d47907c200705194b2d9bbdf307d398f234143de3f291'
tx_index_to_spent = 0
script_pub_key_to_spent = '210377708dd31f718fab3178084a7cf5a8e6bf1e7c4079af5bf9f8c5a0a69a3dd31eac'
amount_to_spent = 8000
receiver_address = "mqFPkGEwujzzz6bPTdco86t3bhpPgDSvSm"
locktime = 0xffffffff

In [12]:
transaction_input = CTxIn(bytes.fromhex(tx_id_to_spent), tx_index_to_spent)
script_pubkey = p2pk_script(public_key_receiver.sec_format())
transaction_output = CTxOut(amount_to_spent, script_pubkey)
